#環境リセット


In [0]:
#colaboratoryのランタイムをリセット
!kill -9 -1

In [0]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}'

#環境構築

In [0]:
!curl https://colab.chainer.org/install | sh -

##CUDA関連

In [0]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb

In [0]:
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb

In [0]:
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub

In [0]:
!apt-get update

In [0]:
!apt-get install cuda

In [0]:
!pip install mxnet-cu92

##gcc -4.8

In [0]:
!gcc -dumpversion

In [0]:
!apt-get install gcc-4.8 g++-4.8

In [0]:
!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-4.8 40 --slave /usr/bin/g++ g++ /usr/bin/g++-4.8

!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-7 50 --slave /usr/bin/g++ g++ /usr/bin/g++-7

In [0]:
!sudo apt-get update

In [0]:
!sudo apt-get upgrade

#Driveにマウント

In [0]:
#Driveのマウントをすることで早く実行
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#your_folderの部分を自分のドライブのフォルダを参照
#darknetがあるフォルダまでカレントディレクトリ
cd drive/My Drive/your_folder

#darknetの処理と学習

In [0]:
!cat darknet/Makefile

In [0]:
cd darknet

In [0]:
%%bash

make

In [0]:
#実行権限を付与
!chmod +x ./darknet

##yoloにて学習

In [0]:
%%bash
#重みのファイルを参照して学習

# v2
#./darknet detector train cfg/obj.data cfg/yolo-obj.data darknet19_448.conv.23 > train_log.txt
# 出力した重み yolo-obj_XXX.weights を読み込む場合
#./darknet detector train cfg/obj.data cfg/yolo-obj.cfg  backup/yolo-obj_XXX.weights > train_log.txt


# v3
./darknet detector train cfg/obj.data cfg/yolo-obj.data darknet53.conv.74 > train_log.txt
#./darknet detector train cfg/obj.data cfg/yolo-obj.data backup/yolo-obj_10000.weights > train_log.txt
#./darknet detector train cfg/obj.data cfg/yolo-obj.data backup/yolo-obj_XXX.weights > train_log.txt

##物体検出の実行

In [0]:
%%bash
#実際に学習された重みを利用して物体検出を行う
#darknet上に検出したい画像をtest.jpgとしていれ実行

# v2
#./darknet detector test cfg/yolo-obj.cfg backup/yolo-obj_XXXX.weights ../test_airport.png
# v3
./darknet detector test cfg/obj.data cfg/yolo-obj.data backup/yolo-obj_XXXX.weights ./test.jpg -thresh 0.002

#バウンディングボックス通りにトリミング

In [0]:
#contentに移動して以下の「細かい料理認識」のソースコードを実行
cd ../../../../

In [0]:
#更に分類するために画像のトリミングを行う
f = open('darknetまでのパス/out_list.txt')
labels = f.read().split('\n')
print(labels)
f.close()

In [0]:
labels.remove("")

In [0]:
for label in range(len(labels)):
  labels[label] = labels[label].strip('[')
  labels[label] = labels[label].strip(']')
  labels[label] = labels[label].strip('\'')
  labels[label] = labels[label].split(",")

In [0]:
labels

In [0]:
for i in range(len(labels)):
  for j in range(len(labels[0])):
    labels[i][j] = labels[i][j].strip()

In [0]:
!mkdir image

In [0]:
from PIL import Image
im = Image.open('./test.jpg')#座標を抜いた元の画像のデータを参照
image = []

In [0]:
#検出されたバウンディングボックス毎に写真をトリミング
for i in range(len(labels)):
  im_crop = im.crop((int(labels[i][0]),int(labels[i][2]),int(labels[i][1]),int(labels[i][3])))
  im_crop.save('./image/out'+str(i)+'.jpg', quality=95)

#細かい料理認識

##インポート

In [0]:
import keras
from keras.applications import VGG16
from keras.models import Sequential, load_model
from keras import models, optimizers
from keras.optimizers import SGD
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.preprocessing.image import load_img,ImageDataGenerator
from keras.callbacks import CSVLogger , ModelCheckpoint
import shutil
import numpy as np
from PIL import Image
import os
import re
from keras.models import load_model
from keras.preprocessing.image import array_to_img, img_to_array
from keras.layers import Dense, Dropout, Activation, Flatten, Input, add
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras import Model
from keras.datasets import cifar10
from keras import Model

In [0]:
def list_pictures(directory, ext='jpg|jpeg|bmp|png|ppm'):#from keras.layers import list_picturesが出来なかったのでここに記述
  return[os.path.join(root, f)
         for root, _, files in os.walk(directory) for f in files
         if re.match(r'([\w]+\.(?:' + ext + '))', f.lower())]

##データ入力のためのディレクトリ作成

In [0]:
mkdir UECFOOD100

In [0]:
mkdir ./UECFOOD100/dataset

In [0]:
mkdir ./UECFOOD100/dataset/{1..100}

In [0]:
uecfood_dir = './UECFOOD100/' #UECFOOD100へのパスの代入
dataset_dir = uecfood_dir + 'dataset/'#datasetディレクトリのパスの代入

##画像データの準備

In [0]:
!wget http://foodcam.mobi/dataset100.zip #UECFOOD100のダウンロード　zip形式

In [0]:
!unzip dataset100.zip

In [0]:
!ls UECFOOD100

###画像のトリミング

In [0]:
#UECFOOD100は指定されたtxtファイルに記載されている座標でトリミングしないと精度が出ないためトリミングを行う
for ctg_index in range(1,100):#1~100番目のディレクトリに実行
  bb_info = []
  for line in open(uecfood_dir + str(ctg_index) +'/bb_info.txt', 'r'):#bb_infoにtxtファイルに記載している各座標を出力
    bb_info.append(line[:-1].split())
  bb_info.pop(0)#0行目はラベル業となっているため削除する
  
  for line in bb_info:#各画像を指定されている座標に合わせてトリミング
      filename = line[0] + '.jpg' #ファイルの名前を各ディレクトリの番号に指定
      pic_img = Image.open(uecfood_dir + str(ctg_index) + '/' + filename)#トリミング実行
      x1 = int(line[1])
      y1 = int(line[2])
      x2 = int(line[3])
      y2 = int(line[4])
      area = (x1,y1,x2,y2)
      pic_crop = pic_img.crop(area)
      #トリミングした画像を別のファイルとして新しく保存する
      pic_crop.save(dataset_dir + str(ctg_index) + '/'+filename,quality=100)
      
      

##UECFOOD100以外の分類したいクラスの追加

In [0]:
#UECFOOD100以外の画像を新たに追加したい場合は100以降のディレクトリに101、102等のディレクトリを追加した後、category_ja_uft.8txt
#に新しく追加したクラスの情報idと料理名を記述

In [0]:
#今回はショートケーキの画像を百前後用意したため新しくショートケーキをクラスに追加

In [0]:
ls

In [0]:
cd ./UECFOOD100/dataset

In [0]:
ls

In [0]:
#ここで新しく追加するクラス101を追加し、その中に画像を保存する

In [0]:
mkdir 101

In [0]:
ls  #101が追加されている

In [0]:
cd ./101

In [0]:
#自身で集めてきた100枚前後の画像を101の中に入れる
from google.colab import files
uploaded = files.upload()

In [0]:
cd ../../

In [0]:
ls

In [0]:
#ローカルでcategory_ja_utf8.txtに新しくクラス情報をを追加したtxtファイルcategory_ja_utf8_pulus.txtをアップロード
from google.colab import files
uploaded = files.upload()

In [0]:
ls

##画像をtest,trainに分類

In [0]:
#/content上で実行
cd ../

In [0]:
#テスト、トレイン用のディレクトリの作成
!mkdir ./UECFOOD100/dataset/train
!mkdir ./UECFOOD100/dataset/test
!mkdir ./UECFOOD100/dataset/train/{1..101}
!mkdir ./UECFOOD100/dataset/test/{1..101}

In [0]:
#訓練データと画像データを分ける
train_dir = dataset_dir + 'train/'
test_dir = dataset_dir + 'test/'

for ctg_index in range(1,101):
  for pic in list_pictures(dataset_dir + str(ctg_index)):
      if np.random.rand(1)<0.2:#ランダムで変数を作成0.2以上ならトレイン、以下ならテストに分ける
         shutil.move(pic, test_dir + str(ctg_index))
      else:
         shutil.move(pic, train_dir + str(ctg_index))

##VGG16モデルで実行

In [0]:
image_size = 224#イメージサイズを設定
 
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))#imagenetから重みを引っ張ってきてモデルを構築
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False
 
model = models.Sequential()
model.add(vgg_conv)
 
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(101, activation='softmax'))
 
model.summary()

###学習のための前処理

In [0]:
#VGGNetを利用する場合はこっちを実行
batch_size =32#バッチサイズの設定
#ImageDataGeneratorを用いて画像の水増し 画像の回転、水平方向垂直方向へのシフト
train_datagen = ImageDataGenerator(
    rescale = 1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip = True
)

validation_datagen=ImageDataGenerator(rescale=1.0/255)
 
train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_size,image_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)
validation_generator=validation_datagen.flow_from_directory(
    test_dir,
    target_size=(image_size,image_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)
 
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

##ResNetで実行

In [0]:
#モデルの構築
def rescell(data, filters, kernel_size, option=False):
    strides=(1,1)
    if option:
        strides=(2,2)
    x=Conv2D(filters=filters,kernel_size=kernel_size,strides=strides,padding="same")(data)
    x=BatchNormalization()(x)
    x=Activation('relu')(x)
    
    data=Conv2D(filters=int(x.shape[3]), kernel_size=(1,1), strides=strides, padding="same")(data)
    x=Conv2D(filters=filters,kernel_size=kernel_size,strides=(1,1),padding="same")(x)
    x=BatchNormalization()(x)
    x=add([x,data])
    x=Activation('relu')(x)
    return x
def ResNet(img_rows, img_cols, img_channels, x_train):
    input=Input(shape=(img_rows,img_cols,img_channels))
    x=Conv2D(32,(7,7), padding="same",activation="relu")(input)
    x=MaxPooling2D(pool_size=(2,2))(x)

    x=rescell(x,64,(3,3))
    x=rescell(x,64,(3,3))
    x=rescell(x,64,(3,3))

    x=rescell(x,128,(3,3),True)

    x=rescell(x,128,(3,3))
    x=rescell(x,128,(3,3))
    x=rescell(x,128,(3,3))

    x=rescell(x,256,(3,3),True)

    x=rescell(x,256,(3,3))
    x=rescell(x,256,(3,3))
    x=rescell(x,256,(3,3))
    x=rescell(x,256,(3,3))
    x=rescell(x,256,(3,3))

    x=rescell(x,512,(3,3),True)

    x=rescell(x,512,(3,3))
    x=rescell(x,512,(3,3))

    x=AveragePooling2D(pool_size=(int(x.shape[1]),int(x.shape[2])),strides=(2,2))(x)

    x=Flatten()(x)
    x=Dense(units=101,kernel_initializer="he_normal",activation="softmax")(x)
    model=Model(inputs=input,outputs=[x])
    return model

###学習の為の前処理

In [0]:
# 入力画像の次元とチャンネル
img_rows, img_cols, img_channels = 224, 224, 3
image_size=224
batch_size = 32
num_classes = 10
epochs = 50

In [0]:
#ImageDataGeneratorを用いて画像の水増し 画像の回転、水平方向垂直方向へのシフト
train_datagen = ImageDataGenerator(
    rescale = 1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip = True
)

validation_datagen=ImageDataGenerator(rescale=1.0/255)
 
train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_size,image_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)
validation_generator=validation_datagen.flow_from_directory(
    test_dir,
    target_size=(image_size,image_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)



In [0]:
model=ResNet(img_rows,img_cols,img_channels, train_generator)
model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
metrics=['accuracy'])

###学習

In [0]:
csv_logger = CSVLogger('food_recongition.csv')#csvファイルに学習の過程、結果を保存
#学習の実行大体10回で40分くらい、100回学習させるとなると6時間程度かかってしまう
hist=model.fit_generator( #学習の実行、step_per_epochとvalidation_stepsの値を変更しなければいけないがステップ数＝全サンプル数÷バッチサイズで出た値を設定する
    train_generator,
    steps_per_epoch=358,#1エポック当たりのステップ数
    epochs=10,#学習回数100回くらいがちょうどええ
    verbose=1,
    validation_data=validation_generator,
    validation_steps=358,
    callbacks=[csv_logger]
)
#重みの保存
model.save('food_recongition.h5')

##結果の表示(グラフ、認識分類結果)

In [0]:
ls#確認 food_recongition.h5 food_recongnition.csvがちゃんとあるかどうか

In [0]:
#グラフの描写
import numpy as np
import matplotlib.pyplot as plt
#csvファイルからデータを取得してnumpy配列に入れる
data = np.genfromtxt("food_recongition.csv",delimiter=",",skip_header=1,)
fig=plt.figure()
ax1 = fig.add_subplot(1, 1, 1)                        
ax1.plot(data.T[3], label="val_acc")
ax1.legend(loc="lower right")

plt.show()

##精度の確認

In [0]:
cd ./content

In [0]:
import requests
from keras.preprocessing import image
 
def download_img(url, file_name):
    r = requests.get(url, stream=True)
    if r.status_code == 200:
        with open(file_name, 'wb') as f:
            f.write(r.content)
 
def get_ctgname_by_index(index):
    ary_ctgname = []
    for line in open('./UECFOOD100/category_ja_utf8_pulus.txt', 'r'):#category_ja_uft.8txtに各料理の名前が入っているので結果をその料理目にで出力増やしたらtxtの中身も変更しなきゃね
        ary_ctgname.append(line[:-1].split())
 
    ctg_dic = train_generator.class_indices
    key = [k for k, v in ctg_dic.items() if v == index]
    id = int(key[0])
    ctg_name_index = 1
    ctg_name = ary_ctgname[int(id)][ctg_name_index]
    return ctg_name

In [0]:
ls

In [0]:
#学習、結果確認の為の前処理を行うことが出来たので制度の結果を確認
from IPython.display import Image,display_jpeg
filename='./out0.jpg'

img = image.load_img('./out0.jpg', target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x / 255.0
pred = model.predict(x)[0]


display_jpeg(Image(filename))
K = 5
unsorted_max_indices = np.argpartition(-pred, K)[:K]
y = pred[unsorted_max_indices]
indices = np.argsort(-y)
top5_indices = unsorted_max_indices[indices]
 
for index in top5_indices:
    ctg_name = get_ctgname_by_index(index)
    print(ctg_name + ":" + str(pred[index]))

In [0]:
#学習、結果確認の為の前処理を行うことが出来たので制度の結果を確認
from IPython.display import Image,display_jpeg
filename='./out1.jpg'

img = image.load_img('./out1.jpg', target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x / 255.0
pred = model.predict(x)[0]


display_jpeg(Image(filename))
K = 5
unsorted_max_indices = np.argpartition(-pred, K)[:K]
y = pred[unsorted_max_indices]
indices = np.argsort(-y)
top5_indices = unsorted_max_indices[indices]
 
for index in top5_indices:
    ctg_name = get_ctgname_by_index(index)
    print(ctg_name + ":" + str(pred[index]))

In [0]:
from IPython.display import Image,display_jpeg
url = 'https://hamsonic.net/wp-content/uploads/2017/09/d45b460cdf65210a1c8dee53858a3522_s.jpg'
filename = 'sample3.jpg'
download_img(url, filename)
 
img = image.load_img(filename, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x / 255.0
pred = model.predict(x)[0]


display_jpeg(Image('sample3.jpg'))
K = 5
unsorted_max_indices = np.argpartition(-pred, K)[:K]
y = pred[unsorted_max_indices]
indices = np.argsort(-y)
top5_indices = unsorted_max_indices[indices]
 
for index in top5_indices:
    ctg_name = get_ctgname_by_index(index)
    print(ctg_name + ":" + str(pred[index]))
